In [ ]:
# default_exp povm

# POVM's

In [ ]:
#hide
import numpy as np
import qutip as qt

from qbism.sics import *
from qbism.utils import *

In [ ]:
#export
def weyl_heisenberg_povm(fiducial):
    return [(1/fiducial.shape[0])*state*state.dag() for state in weyl_heisenberg_states(fiducial)]

In [ ]:
#export
def sic_povm(d):
    return [(1/d)*state*state.dag() for state in sic_states(d)]

In [ ]:
#export
def povm_phi(E):
    phi_inv = np.array([[(a*b/b.tr()).tr() for b in E] for a in E]).real
    return np.linalg.inv(phi_inv)

In [ ]:
#export
def povm_gram(E):
    return np.array([[((a/a.tr())*(b/b.tr())).tr() for b in E] for a in E]).real

In [ ]:
#export
def dm_probs(rho, E):
    return np.array([(e*rho).tr() for e in E]).real

In [ ]:
#export
def probs_dm(p, E, phi=None):
    phi = phi if type(phi) != type(None) else povm_phi(E)
    return sum([c*E[i]/E[i].tr() for i, c in enumerate(phi @ p)])

In [ ]:
#export 
def conditional_probs(A, B):
    return np.array([[((B[i]/B[i].tr())*A[j]).tr() for i in range(len(B))] for j in range(len(A))]).real

In [ ]:
#export 
def tensor_povm(A, B):
    return [qt.tensor(a, b) for b in B for a in A]

In [ ]:
#export
def upgrade_povm(E, i, dims):
    if type(dims) == int:
        dims = [E[0].shape[0]]*dims
    return [qt.tensor(*[e if i == j else qt.identity(d)\
                            for j, d in enumerate(dims)]) for e in E]

In [ ]:
#export
def apply_dims(E, dims):
    for e in E:
        e.dims = [dims, dims]
    return E

In [ ]:
#export
def povm_map(krauss, A, B=None):
    B = B if B else A
    return np.array([[(apply_krauss(a/a.tr(), krauss)*b).tr() for a in A] for b in B]).real

In [ ]:
#export
def implement_povm(E):
    n, d = len(E), E[0].shape[0]
    aux_projectors = [qt.tensor(qt.identity(d), qt.basis(n, i)*qt.basis(n, i).dag()) for i in range(n)]
    V = sum([qt.tensor(E[i].sqrtm(), qt.basis(n, i)) for i in range(n)])
    povm_elements = [V.dag()*aux_projectors[i]*V for i in range(n)]
    assert np.all([np.allclose(E[i], povm_elements[i]) for i in range(n)])
    Q, R = np.linalg.qr(V, mode="complete")
    for i in range(d):
        Q.T[[i,n*i]] = Q.T[[n*i,i]]
    U = qt.Qobj(-Q)
    U.dims = [[d, n],[d, n]]
    assert np.allclose(V, U*qt.tensor(qt.identity(d), qt.basis(n, 0)))
    return U